# LangChain + Chroma 기반 RAG 실습 (sample_docs.txt)

이 노트북에서는 텍스트 파일을 벡터화하여 Chroma DB에 저장하고,  
LangChain을 활용해 RAG(Retrieval-Augmented Generation) 기반 답변 생성을 실습합니다.

---
**실습 목표**
- 텍스트 문서를 로드하고, 검색 효율을 높이기 위해 청크로 분할
- 임베딩 및 벡터 DB(Chroma) 구축
- 유사도 검색을 통한 문맥 기반 답변 생성
- LangChain Prompt 및 RetrievalQA 체인 활용법 익히기

---
**실습 파일:**  
`data/sample_docs.txt` 파일을 대상으로 실습합니다.

In [ ]:
# sample_docs.txt 파일 로드
# TextLoader를 사용해 텍스트 파일을 문서 객체로 불러옵니다.
from langchain_community.document_loaders import TextLoader
import os

data_dir = "../data"
txt_path = os.path.join(data_dir, "sample_docs.txt")

# 명시적으로 encoding을 'utf-8'로 지정
loader = TextLoader(txt_path, encoding='utf-8')
all_documents = loader.load()
print(f"{os.path.basename(txt_path)} 문서 {len(all_documents)}개 로드됨")

## 텍스트 스플리터
- 긴 문서는 LLM 입력 한계 및 검색 효율을 위해 일정 길이의 청크로 분할합니다.
- RecursiveCharacterTextSplitter는 문장, 줄바꿈, 공백 등 다양한 구분자를 활용해 자연스럽게 분할합니다.
- chunk_size, chunk_overlap 값을 조절해 분할 granularity를 실험할 수 있습니다.

In [ ]:
from langchain_core.documents import Document

full_text_content = all_documents[0].page_content

chunks = [Document(page_content=line.strip()) for line in full_text_content.splitlines() if line.strip()]

print(f"총 청크 개수: {len(chunks)}")

# --- 3. 분할된 청크 내용 확인 (모든 청크 출력) ---
print("\n--- 분할된 청크 내용 ---")
for i, chunk in enumerate(chunks):
    print(f'----- 청크 {i+1} -----')
    print(chunk.page_content)
    print() # 청크 간 가독성을 위한 줄바꿈 추가

# Knowledge Base(지식베이스) 구성을 위한 데이터 생성
- 분할된 청크 데이터를 임베딩(벡터화)하여 Chroma DB에 저장합니다.
- 임베딩 모델(OpenAIEmbeddings 등)을 활용해 텍스트를 벡터로 변환합니다.
- persist_directory를 지정하면 DB가 파일로 저장되어 재사용이 가능합니다.

In [13]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴 (.env 파일 필요)
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

from langchain_chroma import Chroma

# Chroma DB에 벡터 저장 (persist_directory로 파일 저장)
database = Chroma.from_documents(documents=chunks, embedding=embedding, collection_name='chroma-txt', persist_directory="../chroma")

# 답변 생성을 위한 Retrieval(검색)
- Chroma에 저장한 벡터 데이터를 유사도 검색(similarity_search)으로 불러옵니다.
- 검색 쿼리는 sample_docs.txt의 내용과 관련된 질문을 사용합니다.
- k 값을 조절해 검색 결과 개수를 실험할 수 있습니다.

In [ ]:
query = 'RAG에 대한 정보 알려줘!'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)
for i, doc in enumerate(retrieved_docs, 1):
    print(f"[{i}] {doc.page_content[:200]}...\n")

# Augmentation을 위한 Prompt 활용
- 검색된 문맥(retrieved_docs)을 LLM에 효과적으로 전달하기 위해 프롬프트 템플릿을 사용합니다.
- LangChain Hub의 `"rlm/rag-prompt"`는 context(검색 결과)와 question(질문)을 받아 답변을 생성하는 표준 RAG 프롬프트입니다.

In [18]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

# 답변 생성 (RetrievalQA 체인)
- RetrievalQA 체인은 검색 결과와 프롬프트를 LLM에 연결해 RAG 파이프라인을 구성합니다.
- query(질문)를 입력하면, 검색 → 프롬프트 조합 → LLM 답변 생성이 자동으로 이루어집니다.

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain.invoke({"query": query})

print(ai_message)